In [16]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Dropout


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [17]:
# load the dataset as time series data
df = pd.read_csv('fifthClean.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
df.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_1580\705661990.py:2: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df = pd.read_csv('fifthClean.csv', header=0, index_col=0, parse_dates=True, squeeze=True)


,Latitude,Longitude,Year,Month,Day,Hour,Minute,Second
0,51.513952,-0.085833,2019,12,1,2,17,49
1,51.511625,-0.081549,2019,12,2,3,14,4
2,51.509775,-0.085014,2019,12,2,4,51,16
3,51.517431,-0.082458,2019,12,3,9,48,8
4,51.516814,-0.081620,2019,12,4,2,53,26


In [28]:
# Create a time series with a time window 
def create_timeseries(data, time_window):
    X = []
    y = []
    for i in range(len(data)-time_window-1):
        t = data[i:(i+time_window)]
        X.append(t)
        y.append(data[i + time_window])
    return np.array(X),np.array(y)

# Create a time series with a time window
time_window = 10
X, y = create_timeseries(df, time_window)

KeyError: 10

In [25]:
print(X)


[[51.513952 51.511625 51.509775 ... 51.51768  51.518262 51.518262]
 [51.511625 51.509775 51.517431 ... 51.518262 51.518262 51.516814]
 [51.509775 51.517431 51.516814 ... 51.518262 51.516814 51.517032]
 ...
 [51.51277  51.513736 51.513736 ... 51.519045 51.513631 51.517239]
 [51.513736 51.513736 51.511794 ... 51.513631 51.517239 51.51119 ]
 [51.513736 51.511794 51.508312 ... 51.517239 51.51119  51.51277 ]]


In [20]:
# reshape y to dim 3
y = y.reshape((y.shape[0], y.shape[1], 1))
print(X.shape, y.shape)

IndexError: tuple index out of range

In [ ]:
# split into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
def plot_mae(history,range,title): #plot the mae
    plt.figure(figsize=(16,8))
    plt.title(title)
    plt.ylabel('Validation mae')
    plt.xlabel('Epochs')
    plt.plot(history['mae'][:range], label='Training mae', color='red')
    plt.plot(history['val_mae'][:range], label='Validation mae', color='green')
    # set x interval to 1
    plt.xticks(np.arange(0, range, 1.0))

    plt.legend(
        loc='upper right',
        shadow=True,
        fontsize='x-large'
    )
    plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.LSTM(32, return_sequences=True))
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.LSTM(64))
model.add(layers.Dense(8))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

# fit model
history = model.fit(X_train, y_train, epochs=1024, verbose=1, validation_data=(X_test, y_test),batch_size=512)

# plot the mae
plot_mae(history.history,1024,'LSTM')

